In [ ]:
!pip3 install monobank pandas

In [ ]:
import monobank
import pandas as pd
from datetime import date, timedelta
import time

# 3. Initialize the client with your token
token = 'token'
mono = monobank.Client(token)

In [ ]:
mono.get_client_info()

In [ ]:
def get_statements_as_df(account_id: str, start_date: date) -> pd.DataFrame:
    """
    Retrieve all operations from a given date to today for a specific account 
    and return them as a pandas DataFrame.

    Parameters:
    account_id (str): The account ID to retrieve statements for.
    start_date (date): The start date to retrieve statements from.

    Returns:
    pd.DataFrame: A DataFrame containing the account statements.
    """
    
    end_date = date.today()
    current_start_date = start_date
    statements = []
    
    # Fetch data in 30-day intervals
    while current_start_date <= end_date:
        # Make sure 'to' date is not more than 30 days from 'from' date and it's not a future date
        current_end_date = min(current_start_date + timedelta(days=30), end_date, date.today())
        try:
            statements += mono.get_statements(account_id, current_start_date, current_end_date)
        except monobank.TooManyRequests:
            print("Too many requests. Waiting for 60 seconds before retrying.")
            time.sleep(60)  # wait for 60 seconds before retrying
            continue
        current_start_date = current_end_date + timedelta(days=1)

    # Convert the statements into a DataFrame
    df = pd.DataFrame(statements)
    
    return df


In [ ]:
mono.get_statements('token',  date(2022,10,4),  date(2022,10,6))

In [ ]:
history = get_statements_as_df('token', date(2022,10,4))

In [ ]:
def divide100(x):
    return x/100
# apply to 'amount' 'operationAmount' 'cashbackAmount' 'balance' 
df['amount'] = df['amount'].apply(divide100)
df['operationAmount'] = df['operationAmount'].apply(divide100)
df['cashbackAmount'] = df['cashbackAmount'].apply(divide100)
df['balance'] = df['balance'].apply(divide100)

df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

df.to_csv('monobank.csv')